## Overview
The Hipparcos mission obtained magnitude data for 118K stars beginning in 1989. The Gaia mission was launched in 2013 to gather data on about a billion objects. We model one of the Hipparcos magnitudes as a function of those from Gaia DR2 and derive an approximation of magnitude change in ~25 years.

We also model the standard error of the magnitude change. A *mag_change_anomaly* metric is provided along with other results in the Output section of this notebook.

## The dataset
We will use a Kaggle dataset titled _[79K Gaia DR2 Stars Crossmatched With Hipparcos](https://www.kaggle.com/solorzano/79k-gaia-dr2-stars-crossmatched-with-hipparcos)_. The number of rows in the dataset is:

In [ ]:
import pandas as pd

work_data = pd.read_csv('/kaggle/input/79k-gaia-dr2-stars-crossmatched-with-hipparcos/hipparcos-gaia-data.csv')
len(work_data)

## Exclusion of distant stars
We're only interested in stars that are within 500 parsecs of the sun. Limiting distance prior to modeling also improves model accuracy. The resulting number of rows is now:

In [ ]:
work_data = work_data[work_data['parallax'] >= 2.0].reset_index(drop=True)
len(work_data)

## Modeling helper functions
We use a function named *get_cv_model_transform* to do model training using an arbitrary Machine Learning algorithm. The function averages multiple runs of k-fold cross-validation, and produces a transform function that can be applied to any dataframe with the required variables. Model results are "out of bag" (i.e. not overfitted to the training data.)

In [ ]:
import types
import numpy as np
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler 

warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(2019050001)

def get_cv_model_transform(data_frame, label_extractor, var_extractor, trainer_factory, response_column='response', 
                           id_column='source_id', n_runs=2, n_splits=2, max_n_training=None, scale=False,
                           trim_fraction=None, classification=False):
    '''
    Creates a transform function that results from training an arbitrary regression model with k-fold
    cross-validation. Multiple runs are averaged out. The transform function takes a frame and adds a 
    response column to it. Results are "out of bag" (not overfitted).
    '''
    default_model_list = []
    sum_series = pd.Series([0] * len(data_frame)).astype(float)
    for r in range(n_runs):
        shuffled_frame = data_frame.sample(frac=1)
        shuffled_frame.reset_index(inplace=True, drop=True)
        response_frame = pd.DataFrame(columns=[id_column, 'response'])
        kf = KFold(n_splits=n_splits)
        first_fold = True
        for train_idx, test_idx in kf.split(shuffled_frame):
            train_frame = shuffled_frame.iloc[train_idx]
            if trim_fraction is not None:
                helper_labels = label_extractor(train_frame) if isinstance(label_extractor, types.FunctionType) else train_frame[label_extractor] 
                train_label_ordering = np.argsort(helper_labels)
                orig_train_len = len(train_label_ordering)
                head_tail_len_to_trim = int(round(orig_train_len * trim_fraction * 0.5))
                assert head_tail_len_to_trim > 0
                trimmed_ordering = train_label_ordering[head_tail_len_to_trim:-head_tail_len_to_trim]
                train_frame = train_frame.iloc[trimmed_ordering]
            if max_n_training is not None:
                train_frame = train_frame.sample(max_n_training)
            train_labels = label_extractor(train_frame) if isinstance(label_extractor, types.FunctionType) else train_frame[label_extractor]
            test_frame = shuffled_frame.iloc[test_idx]
            train_vars = var_extractor(train_frame)
            test_vars = var_extractor(test_frame)
            scaler = None
            if scale:
                scaler = StandardScaler()  
                scaler.fit(train_vars)
                train_vars = scaler.transform(train_vars)  
                test_vars = scaler.transform(test_vars) 
            trainer = trainer_factory()
            fold_model = trainer.fit(train_vars, train_labels)
            test_responses = fold_model.predict_proba(test_vars)[:,1] if classification else fold_model.predict(test_vars)
            test_id = test_frame[id_column]
            assert len(test_id) == len(test_responses)
            fold_frame = pd.DataFrame({id_column: test_id, 'response': test_responses})
            response_frame = pd.concat([response_frame, fold_frame], sort=False)
            if first_fold:
                first_fold = False
                default_model_list.append((scaler, fold_model,))
        response_frame.sort_values(id_column, inplace=True)
        response_frame.reset_index(inplace=True, drop=True)
        assert len(response_frame) == len(data_frame), 'len(response_frame)=%d' % len(response_frame)
        sum_series += response_frame['response']
    cv_response = sum_series / n_runs
    assert len(cv_response) == len(data_frame)
    assert len(default_model_list) == n_runs
    response_map = dict()
    sorted_id = np.sort(data_frame[id_column].values) 
    for i in range(len(cv_response)):
        response_map[str(sorted_id[i])] = cv_response[i]
    response_id_set = set(response_map)
    
    def _transform(_frame):
        _in_trained_set = _frame[id_column].astype(str).isin(response_id_set)
        _trained_frame = _frame[_in_trained_set].copy()
        _trained_frame.reset_index(inplace=True, drop=True)
        if len(_trained_frame) > 0:
            _trained_id = _trained_frame[id_column]
            _tn = len(_trained_id)
            _response = pd.Series([None] * _tn).astype(float)
            for i in range(_tn):
                _response[i] = response_map[str(_trained_id[i])]
            _trained_frame[response_column] = _response
        _remain_frame = _frame[~_in_trained_set].copy()
        _remain_frame.reset_index(inplace=True, drop=True)
        if len(_remain_frame) > 0:
            _unscaled_vars = var_extractor(_remain_frame)
            _response_sum = pd.Series([0] * len(_remain_frame)).astype(float)
            for _model_tuple in default_model_list:
                _scaler = _model_tuple[0]
                _model = _model_tuple[1]
                _vars = _unscaled_vars if _scaler is None else _scaler.transform(_unscaled_vars)
                _response = _model.predict_proba(_vars)[:,1] if classification else _model.predict(_vars)
                _response_sum += _response
            _remain_frame[response_column] = _response_sum / len(default_model_list)
        _frames_list = [_trained_frame, _remain_frame]
        _result = pd.concat(_frames_list, sort=False)
        _result.reset_index(inplace=True, drop=True)
        return _result
    
    return _transform

import scipy.stats as stats

def print_evaluation(data_frame, label_column, response_column):
    '''
    Compares a label with a model response and prints RMSE and correlation statistics.
    '''
    response = response_column(data_frame) if isinstance(response_column, types.FunctionType) else data_frame[response_column]
    label = label_column(data_frame) if isinstance(label_column, types.FunctionType) else data_frame[label_column]
    residual = label - response
    rmse = np.sqrt(sum(residual ** 2) / len(data_frame))
    correl = stats.pearsonr(response, label)[0]
    print('RMSE: %.5f | Correlation: %.4f' % (rmse, correl,), flush=True)

## Base model
We will start with a simple linear model. The variables will be the 3 magnitude bands made available by Gaia DR2, plus parallax distance in kilo-parsecs.

In [ ]:
def extract_vars(data_frame):
    return np.transpose([
        1.0 / data_frame['parallax'],
        data_frame['phot_g_mean_mag'],
        data_frame['phot_bp_mean_mag'],
        data_frame['phot_rp_mean_mag'],
    ])

The training label will be <code>hpmag</code>, which is the median Hipparcos magnitude. This is the magnitude that seems to produce the most accurate model. Model evaluation follows:

In [ ]:
from sklearn.linear_model import LinearRegression

LABEL_COLUMN = 'hpmag'

def extract_label(data_frame):
    return data_frame[LABEL_COLUMN]


def get_base_trainer():
    return LinearRegression()


base_transform = get_cv_model_transform(work_data, extract_label, extract_vars, get_base_trainer, 
        n_runs=3, n_splits=5, max_n_training=None, response_column='base_response' , scale=False)
work_data = base_transform(work_data)
print_evaluation(work_data, LABEL_COLUMN, 'base_response')

## Non-linear model of residuals

Dataset magnitudes are in different bands, and much of the error could be due to spectrophotometric peculiarities of different stars (in addition to crossmatching error, systematics and so forth.)

In order to deal with spectrophotometric error, we will train a Neural Network that models the linear residual as a function of color features (i.e. differences between magnitudes of different bands in each dataset.) Note that no cross-database differences should be used as variables of this model, for obvious reasons.

In [ ]:
from sklearn.neural_network import MLPRegressor

def base_residual_transform(data_frame):
    new_frame = data_frame.copy()
    new_frame['base_residual'] = new_frame[LABEL_COLUMN] - new_frame['base_response']
    return new_frame


def extract_res_vars(data_frame):
    g_mag = data_frame['phot_g_mean_mag']
    bp_mag = data_frame['phot_bp_mean_mag']
    rp_mag = data_frame['phot_rp_mean_mag']
    btmag = data_frame['btmag']
    hpmag = data_frame['hpmag']
    vmag = data_frame['vmag']
    vtmag = data_frame['vtmag']
    return np.transpose([
        g_mag - bp_mag, 
        rp_mag - g_mag,
        btmag - vmag,
        vmag - vtmag,
        hpmag - vmag,
        hpmag - vtmag,
        btmag - vtmag,
    ])


def extract_res_label(data_frame):
    return data_frame['base_residual']


def get_res_trainer():
    return MLPRegressor(hidden_layer_sizes=(20), max_iter=400, alpha=0.1, random_state=np.random.randint(1,10000))


work_data = base_residual_transform(work_data)
res_transform = get_cv_model_transform(work_data, extract_res_label, extract_res_vars, get_res_trainer, 
        n_runs=3, n_splits=3, max_n_training=None, response_column='modeled_residual' , scale=True)
work_data = res_transform(work_data)
print_evaluation(work_data, 'base_residual', 'modeled_residual')

This is a good improvement over the linear model (~34%).

## Preliminary magnitude change estimate
The difference between the residual of the base linear model and the modeled residual is an approximation of the magnitude change from Gaia to Hipparcos. We want the negative of this (Hipparcos to Gaia).

In [ ]:
def mag_change_transform(data_frame):
    new_frame = data_frame.copy()
    new_frame['prelim_mag_change_estimate'] = new_frame['modeled_residual'] - new_frame['base_residual']
    return new_frame


work_data = mag_change_transform(work_data)

## Magnitude bias analysis
We've looked at possible model biases in distance, galactic latitude, celestial declination and magnitude. With magnitude, there does appear to be a bias that should be corrected. Below is a binned line plot of Gaia G magnitude vs. the estimated magnitude change from Hipparcos to Gaia.

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
py.init_notebook_mode(connected=False)

def line_plot(bins_x, bins_y, lower_y=None, upper_y=None, x_range=None, x_label='', y_range=None, y_label='', title=''):
    trace1 = go.Scatter(
        x=bins_x,
        y=bins_y,
        mode='lines',
        line=dict(width=4),
        marker=dict(
            size=3,
        ),
        showlegend=False
    )
    scatter_data = [trace1]
    if lower_y is not None and upper_y is not None:
        scatter_data.append(go.Scatter(
            x=bins_x,
            y=upper_y,
            marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            showlegend=False
        ))
        scatter_data.append(go.Scatter(
            x=bins_x,
            y=lower_y,
            marker=dict(color="#444"),
            line=dict(width=0),
            mode='lines',
            fillcolor='rgba(68, 68, 68, 0.1)',
            fill='tonexty',
            showlegend=False
        ))
        
    
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title=x_label,
            range=x_range,
        ),
        yaxis=dict(
            title=y_label,
            range=y_range,
        ),
    )
    fig = go.Figure(data=scatter_data, layout=layout)
    py.iplot(fig)

    
def plot_binned(data_frame, x_column, y_column, step=0.1, x_range=None, x_label='', y_range=None, y_label='', title=''):
    x = data_frame[x_column].astype(float)
    factor = 1.0 / step
    bin_x = np.round(x * factor) / factor
    work_frame = data_frame.assign(__bin_x = bin_x)
    groups = work_frame.groupby('__bin_x')
    bins_x = []
    bins_y = []
    lower_y = []
    upper_y = []
    for group in groups:
        key = group[0]
        group_frame = group[1]
        len_gf = len(group_frame)
        if len_gf >= 3:
            values = group_frame[y_column].values
            mean_value = np.mean(values)
            std_value = np.std(values) / np.sqrt(len_gf)
            upper_bound = mean_value + std_value * 1.96
            lower_bound = mean_value - std_value * 1.96
            bins_x.append(key)
            bins_y.append(mean_value)
            lower_y.append(lower_bound)
            upper_y.append(upper_bound)
    line_plot(bins_x, bins_y, upper_y=upper_y, lower_y=lower_y, x_range=x_range, 
              x_label=x_label, y_range=y_range, y_label=y_label, title=title)
    
    
plot_binned(work_data, 'phot_g_mean_mag', 'prelim_mag_change_estimate', 
            x_range=None, x_label='Gaia G magnitude', step=0.05,
            y_range=[-0.2,0.2], y_label='Preliminary magnitude change estimate',
            title='Magnitude (binned) vs. magnitude change estimate')

Distance (below) does not have nearly as much bias, and it has much less variance, even though there's plenty of uncertainty in parallax observations. This can be explained by observing that any any given distance in the dataset, there are stars of different luminosity.

In [ ]:
work_data['distance'] = 1000.0 / work_data['parallax']
plot_binned(work_data, 'distance', 'prelim_mag_change_estimate', 
            x_range=None, x_label='Parallax distance (Parsecs)', step=5,
            y_range=[-0.05,0.05], y_label='Preliminary magnitude change estimate',
            title='Distance (binned) vs. magnitude change estimate')

## Position bias analysis
We have observed that stellar datasets, particularly those produced using observations from the ground, contain line-of-sight distortions. We will model the preliminary magnitude change estimate as a function of Right Ascension and Declination coordinates, using a Random Forest, to determine what we're dealing with.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def extract_posbias_vars(data_frame):
    ra = np.deg2rad(data_frame['ra'].values)
    dec = np.deg2rad(data_frame['dec'].values)
    return np.transpose([ra, dec])    


def get_posbias_trainer():
    return RandomForestRegressor(n_estimators=60, max_depth=18, min_samples_split=30, random_state=np.random.randint(1,10000))


transform_posbias = get_cv_model_transform(work_data, 'prelim_mag_change_estimate', extract_posbias_vars, get_posbias_trainer, 
        n_runs=3, n_splits=2, max_n_training=None, response_column='modeled_posbias', scale=False,
        trim_fraction=0.003)
work_data = transform_posbias(work_data)
print_evaluation(work_data, 'prelim_mag_change_estimate', 'modeled_posbias')

Position-dependent biases are depicted below in a color-coded scatter chart.

In [ ]:
def scatter_plot(bins_x, bins_y, colors, x_range=None, x_label='', y_range=None, y_label='', title='', colorbar_title=''):
    trace1 = go.Scatter(
        x=bins_x,
        y=bins_y,
        mode='markers',
        marker=dict(
            size=5,
            color=colors,
            colorbar=dict(
                title=colorbar_title
            ),
        ),
        showlegend=False
    )
    scatter_data = [trace1]
    layout = go.Layout(
        title=title,
        xaxis=dict(
            title=x_label,
            range=x_range,
        ),
        yaxis=dict(
            title=y_label,
            range=y_range,
        ),
    )
    fig = go.Figure(data=scatter_data, layout=layout)
    py.iplot(fig)

    
sample_data = work_data.sample(n=10000)
scatter_plot(sample_data['ra'], sample_data['dec'], sample_data['modeled_posbias'],
             x_label='Right ascension (degrees)', y_label='Declination (degrees)', colorbar_title='Modeled residual',
             title='Responses of model of blend residual<br>by position in the sky')

There are some indications of bias in certain places, but this mostly looks like noise. Just to make sure there aren't complex interactions/systematics between distance or magnitude and position in the sky, we will get two sets of a thousand outliers at the two ends of the modeled position bias, and analyze how they behave.

The following plot shows magnitude vs. estimated magnitude change for "high" outliers.

In [ ]:
pos_set_high = work_data.sort_values('modeled_posbias', ascending=False).head(1000)
pos_set_low = work_data.sort_values('modeled_posbias', ascending=True).head(1000)

plot_binned(pos_set_high, 'phot_g_mean_mag', 'prelim_mag_change_estimate', 
            x_range=None, x_label='Gaia G magnitude', step=0.1,
            y_range=[-0.2,0.2], y_label='Preliminary magnitude change estimate',
            title='Magnitude (binned) vs. magnitude change estimate<br>' +
            'for "high" outliers in position-based model')

This is magnitude vs. magnitude change for "low" outliers:

In [ ]:
plot_binned(pos_set_low, 'phot_g_mean_mag', 'prelim_mag_change_estimate', 
            x_range=None, x_label='Gaia G magnitude', step=0.1,
            y_range=[-0.2,0.2], y_label='Preliminary magnitude change estimate',
            title='Magnitude (binned) vs. magnitude change estimate<br>' +
            'for "low" outliers in position-based model')

In both outlier datasets, magnitude bias is consistent with what we see in the whole dataset. We will not correct for position biases. While model accuracy would improve a little, the trade-off is that we would lose some position-based information regarding regional dimming or brightening.

## Magnitude bias correction

We will attempt to correct the magnitude-dependent bias with a Neural Network. We will use Gaia magnitudes as variables of the correction model. Hipparcos magnitudes should not be used as variables, as they would leak information, given that our original label is a Hipparcos magnitude. This is true regardless of the fact that we're only modeling a residual of the original model.

We include parallax distance as a variable of the model as well. A magnitude correction could introduce distance-based distortions.

In [ ]:
def extract_mbias_vars(data_frame):
    return data_frame[['phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'distance']]


def get_mbias_trainer():
    return MLPRegressor(hidden_layer_sizes=(15), max_iter=500, alpha=0.1, random_state=np.random.randint(1,10000))


mbias_transform = get_cv_model_transform(work_data, 'prelim_mag_change_estimate', extract_mbias_vars, get_mbias_trainer, 
        n_runs=5, n_splits=3, max_n_training=None, response_column='mbias_response', scale=True, 
        trim_fraction=0.005)
work_data = mbias_transform(work_data)
print_evaluation(work_data, 'prelim_mag_change_estimate', 'mbias_response')

After correction, this is what the magnitude vs. magnitude change plot looks like:

In [ ]:
work_data['mag_change_estimate'] = work_data['prelim_mag_change_estimate'] - work_data['mbias_response']
plot_binned(work_data, 'phot_g_mean_mag', 'mag_change_estimate', 
            x_range=None, x_label='Gaia G magnitude', step=0.05,
            y_range=[-0.2,0.2], y_label='Corrected magnitude change estimate',
            title='Magnitude (binned) vs. magnitude change estimate')

We also get a slight correction of distance-dependent biases:

In [ ]:
plot_binned(work_data, 'distance', 'mag_change_estimate', 
            x_range=None, x_label='Parallax distance (Parsecs)', step=5,
            y_range=[-0.05,0.05], y_label='Corrected magnitude change estimate',
            title='Distance (binned) vs. magnitude change estimate')

## Distribution and outliers
Let's look at summary statistics of the *mag_change_estimate* metric (approx. magnitude change in ~25 years).

In [ ]:
work_data['mag_change_estimate'].astype(float).describe()

The distribution is quite skewed right, as we can see in the following histogram, where we've truncated the y axis.

In [ ]:
def plot_distribution(x, xlabel, xrange=None, yrange=None, xbins=None, lineX=None, title=None):
    hist_trace = go.Histogram(
        x=x,
        xbins=xbins,
    )
    data = [hist_trace]
    shapes = None
    if lineX is not None:
        shapes = [
            { 
                'type': 'line', 'yref': 'paper', 'x0': lineX, 'x1': lineX, 'y0': 0, 'y1': 1.0, 
                'line': { 'color': 'orange', 'width': 3, 'dash': 'dashdot',},
            }
        ]
    layout = go.Layout(barmode='overlay', 
        title=title,
        xaxis=dict(
            title=xlabel,
            range=xrange,
        ),
        yaxis=dict(
            title='Frequency',
            range=yrange,
        ),
        shapes=shapes,
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
    
    
plot_distribution(work_data['mag_change_estimate'], 'Magnitude change', xrange=[-0.5,0.8], yrange=[0,200], title='Distribution of approx. magnitude change in ~25 years')

Let's get a set of 3-sigma dimmers and check the summary statistics of their proper motions.

In [ ]:
anom_threshold = np.std(work_data['mag_change_estimate']) * 3
extreme_dimmers = work_data[work_data['mag_change_estimate'] >= anom_threshold]
comparison_sample = work_data.sample(n=len(extreme_dimmers))

ed_pm = np.sqrt(extreme_dimmers['pmra'] ** 2 + extreme_dimmers['pmdec'] ** 2)
cs_pm = np.sqrt(comparison_sample['pmra'] ** 2 + comparison_sample['pmdec'] ** 2)

ed_pm.describe()

How does a random sample of the same size compare?

In [ ]:
cs_pm.describe()

Proper motion of the random sample is slightly higher, so proper motion doesn't explain the dim outliers.

We can also look at summary statistics of the Gaia flux error of the extreme dimmers.

In [ ]:
ed_ef = extreme_dimmers['phot_g_mean_flux_error'] / extreme_dimmers['phot_g_mean_flux']
cs_ef = comparison_sample['phot_g_mean_flux_error'] / comparison_sample['phot_g_mean_flux']

ed_ef.describe()

How does it compare to a random sample?

In [ ]:
cs_ef.describe()

In this case, there's an evident issue. And it shouldn't be surprising. The higher the Gaia flux error, the less certainty we should have about the magnitude change estimate. If we were calculating a linear regression slope, we would want to estimate the standard slope error. The more noisy the data, the higher the standard error. We need to do something similar in this case, i.e. model the error.

## Squared magnitude change (i.e. squared error) modeling
We will model the square of the magnitude change estimate as a function of the Gaia magnitude error, the parallax error, and the Gaia flux, using a Random Forest.

It's not enough to model the the squared residual so far, as that residual could have biases in the variable space we're using to train the error model. So we first train for bias using the same variable space and training algorithm that we will use to model squared residuals.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def extract_error_vars(data_frame):
    flux_error = data_frame['phot_g_mean_flux_error']
    flux = data_frame['phot_g_mean_flux']
    gaia_error_feature = np.log((flux + flux_error) / flux) ** 2
    
    return np.transpose([
        gaia_error_feature,
        data_frame['parallax_error'],
        flux,
    ])


def get_residual_trainer():
    return RandomForestRegressor(n_estimators=100, max_depth=8, min_samples_split=10, random_state=np.random.randint(1,10000))


transform_em_bias = get_cv_model_transform(work_data, 'mag_change_estimate', extract_error_vars, get_residual_trainer, 
        n_runs=4, n_splits=2, response_column='expected_em_mc', scale=False,
        trim_fraction=None)
work_data = transform_em_bias(work_data)
print_evaluation(work_data, 'mag_change_estimate', 'expected_em_mc')

This bias model improves our overall accuracy.

Now we calculate a new model residual, and then we model its square. Because we're using a Random Forest, responses of the error model can be thought of as regional mean squares of deviations from the model, in the variable space.

In [ ]:
def get_squared_mag_change(data_frame):
    return (data_frame['mag_change_estimate'] - data_frame['expected_em_mc']) ** 2

transform_expected_mc_sq = get_cv_model_transform(work_data, get_squared_mag_change, extract_error_vars, get_residual_trainer, 
        n_runs=4, n_splits=2, response_column='expected_mc_sq', scale=False,
        trim_fraction=None)
work_data = transform_expected_mc_sq(work_data)
print_evaluation(work_data, get_squared_mag_change, 'expected_mc_sq')

We will add some columns to the dataset:
* *mag_change_std_error*: The square root of the modeled square error (similar to an RMSE or standard deviation.)
* *updated_mag_change_estimate*: The new magnitude change due to the bias modeled in the variable space of the error model.
* *mag_change_anomaly*: The new magnitude change estimate divided by the modeled standard error.

In [ ]:
work_data = work_data.assign(mag_change_std_error = np.sqrt(work_data['expected_mc_sq'].astype(float)))
work_data = work_data.assign(updated_mag_change_estimate = work_data['mag_change_estimate'] - work_data['expected_em_mc'])
work_data = work_data.assign(mag_change_anomaly = work_data['updated_mag_change_estimate'] / work_data['mag_change_std_error'])

The standard deviation of the *mag_change_anomaly* metric should theoretically be 1.0, provided there are no extreme outliers.

In [ ]:
np.std(work_data['mag_change_anomaly'])

## Output file
We now dump model results to a file, with the following columns:
* *source_id*: The Gaia DR2 ID of the star.
* *mag_change_est_linear*: The magnitude change estimated with the simple linear model.
* *mag_change_est*: The modeled magnitude change from Hipparcos to Gaia.
* *mag_change_std_error*: The square root of the modeled square error.
* *mag_change_anomaly*: The magnitude change estimate divided by the modeled standard error.

In [ ]:
saved_data = pd.DataFrame({
    'source_id': work_data['source_id'],
    'mag_change_est_linear': -work_data['base_residual'],
    'mag_change_est': work_data['updated_mag_change_estimate'],
    'mag_change_std_error': work_data['mag_change_std_error'],
    'mag_change_anomaly': work_data['mag_change_anomaly'],
})

saved_data.round(4).to_csv('hipparcos-gaia-mag-change-estimate.csv', index=False)

## Acknowledgements

This work has made use of data from the European Space Agency (ESA) mission Gaia (https://www.cosmos.esa.int/gaia), processed by the Gaia Data Processing and Analysis Consortium (DPAC, https://www.cosmos.esa.int/web/gaia/dpac/consortium). Funding for the DPAC has been provided by national institutions, in particular the institutions participating in the Gaia Multilateral Agreement.

The Hipparcos and Tycho Catalogues are the primary products of the European Space Agency's astrometric mission, Hipparcos. The satellite, which operated for four years, returned high quality scientific data from November 1989 to March 1993.